# Presentación 

**Alumno**: Arevalo Ezequiel Matias 

**DNI**: 34520876

**Comision**: 3

**Ciudad a analizar**: Atenas (Grecia)

# Descarga del dataset.

In [1]:
import requests
import gzip 
import shutil
from os.path import exists

CONFIG={
    "data_path": "./listings.csv",
    "data_url": "http://data.insideairbnb.com/greece/attica/athens/2022-06-17/data/listings.csv.gz"
}

def uncompress_data(source_filename, dest_filename=CONFIG["data_path"]):
    
    with gzip.open(source_filename, "rb") as f_in:
        with open(dest_filename, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
            return dest_filename

def download_data(url=CONFIG["data_url"], local_filename=CONFIG["data_path"], overwrite=False, uncompress=True):

    d_file=url.split('/')[-1]

    if exists(local_filename) and overwrite is False:
        print (f"{local_filename} already exists. Set overwrite to True to force replacement.")
        return local_filename

    try:
        with requests.get(url, stream=True) as r:
                r.raise_for_status()
                with open(d_file, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192): 
                        f.write(chunk)
                print( f"{d_file} Download complete")

    except Exception as e:
        print( str(e) )
        return None 
    
    try:
        if uncompress:
            uncompress_data(d_file, local_filename)
            print (f"{d_file} created. Refresh your panel to see the file in the folder tree.")
    except Exception as e:
        print( str(e) )

    return local_filename if uncompress else d_file

download_data()

listings.csv.gz Download complete
listings.csv.gz created. Refresh your panel to see the file in the folder tree.


'./listings.csv'

In [2]:
# Definicion de funciones y filtros para la creacion de la estructura de datos

In [3]:
import csv

def string_to_float(value):
      ''' Limpia el formato de un numero de una celda
          en formato string, convirtiendolo a flotante '''   
      return float( value.replace('$', '').replace(',', '') )

def avarage(l, n_decimal=2 ):
    ''' Calcula el promedio de una lista '''
    return round(sum(l) / len(l), n_decimal)

def count_elements(l):
    ''' toma una lista, y retorna un diccionario
        que asocia a cada elemento de la lista
        el numero de veces que este aparece en la lista 
    '''
    result = {}

    for i in l: 
        if i in result:
            result[i]+=1
        else:
            result[i]=1
    return result
    
def create_dict(as_key, as_value, map_on_values=None):
    ''' as_key=<String>
        as_value=<String>
        map_on_values=[function]

        as_key: Toma como valor un string con el nombre de la columna a utilizar para filtrar los valores que seran _
        utilizados como keys.

        as_value: Toma como valor un string con el nombre de la columna que se va a utilziar para filtrar los valores _
        que seran concatenados a una lista y asignados las disntas keys procesadas.

        map_on_values: Toma una funcion que sera usada para formatear los valores antes de ser asignados al array utilizado _
        como filtro de valores.
    '''

    with open(CONFIG["data_path"], newline='', encoding="utf8") as f:

        d = {}
        reader = csv.DictReader(f, delimiter=',')
        
        for row in reader:
            key = row[ as_key ]
            
            if  key not in d.keys():
                d[ key ] = []
            
            value = map_on_values(row[as_value]) if map_on_values else row[as_value]
            d[ key ].append( value )
        
        return d

# Tratamiento de los datos y creacion de las estructuras de diccionarios.

In [4]:
'''
Se obtienen todos los precios de un barrio y se calcula su promedio.
El diccionario de los promedios se calcula usando diccionarios por comprension.
'''
precio_x_barrio = create_dict("neighbourhood_cleansed", "price", map_on_values=string_to_float ) 
promedio_precio_x_barrio= {  nc: avarage(p)  for nc,p in precio_x_barrio.items()  }


'''
Se obtienen todos los tipos de habitacion por barrio.
Se crea una nueva estructura que expande al diccionario anterior, guardando por cada barrio:
todos los tipos de habitaciones, el total de los distintos tipos de habitacioens, la cantidad
de habitaciones de cada tipo y el porcentaje de oferta de cada tipo en el barrio.

Estructura :
expan_tipo_habitacion_x_barrio={
  neighbourhood : {
    "habitaciones" : [ list ],
    "total": int,
    "cantidad_habitacion_x_tipo": { 
      "tipo habitacion": int(cantidad)
    },
    "porcentaje_habitacion_x_tipo": {
       "tipo habitacion": float(porcentaje)
    }
  }
}
'''

tipo_habitacion_x_barrio = create_dict("neighbourhood_cleansed", "room_type")
expan_tipo_habitacion_x_barrio = {}
for neighbourhood,room_types in tipo_habitacion_x_barrio.items():
    expan_tipo_habitacion_x_barrio[neighbourhood]={}
    expan_tipo_habitacion_x_barrio[neighbourhood]["habitaciones"]=room_types
    expan_tipo_habitacion_x_barrio[neighbourhood]["total"]=len(room_types)
    expan_tipo_habitacion_x_barrio[neighbourhood]["cantidad_habitacion_x_tipo"]=count_elements(room_types)

    expan_tipo_habitacion_x_barrio[neighbourhood]["porcentaje_habitacion_x_tipo"] = {  
        rt: (x*100)/len(room_types)  for rt,x in count_elements(room_types).items()  
    }

# Analisis de los datos y despliegue de los graficos.

## Importacion de los modules para graficar

In [5]:

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

## Precio promedio de los alquileres disponibles en cada barrio de Atenas

In [ ]:

def create_bar(dictio, xlabel=None, ylabel=None, title=None):
    plt.figure(figsize=(10, 8))
    plt.xticks(rotation=90, fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel(xlabel, fontsize=10)
    plt.ylabel(ylabel, fontsize=10)
    plt.title(title, fontsize=20)
    plt.bar(dictio.keys(), dictio.values())        

create_bar(promedio_precio_x_barrio, 'barrios', 'precio promedio', 'precio promedio por barrio')

plt.show()


## Porcentaje de ofertas de cada tipo de alojamiento sobre el total de ofertas de cada barrio

In [ ]:

def create_pie(clave, dictio, name=None):
    plt.figure(figsize=(3, 3))
    plt.title(name, fontsize=10)
    plt.pie(
        dictio.values(), 
        labels=dictio.keys()
        ,autopct='%.0f%%',
        textprops={'size': 'smaller'},
        shadow=True, radius=1
    )

for barrio, values in expan_tipo_habitacion_x_barrio.items():
    create_pie(barrio, values["porcentaje_habitacion_x_tipo"], "tipos de habitaciones en {0}".format(barrio))

plt.show()